In [6]:
import importData as import_data
import torch
import torch.nn as nn
import torch.optim as optim
import os
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.model_selection import train_test_split
import numpy as np
id = import_data.importData()

# Define the model
class CNNLSTMModel(nn.Module):
    def __init__(self, max_seq_length):
        super(CNNLSTMModel, self).__init__()
        self.conv = nn.Conv2d(10, 128, kernel_size=(3, 1))
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=(2, 1))  # Only pool along the height dimension
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(128 * 499, 128)  # Adjusted dimensions after conv and pool
        self.repeat = nn.Linear(128, 128 * max_seq_length)
        self.lstm = nn.LSTM(128, 64, batch_first=True)
        self.fc_out = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.relu(self.fc(x))
        x = self.repeat(x)
        x = x.view(-1, max_seq_length, 128)  # Reshape to (batch_size, seq_length, input_size)
        x, _ = self.lstm(x)
        x = self.fc_out(x)
        return x
    

testuuid = "98006881-B2F0-4B6B-94CF-A21ABE69F21B"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

d_test, t_test = id.import_data(testuuid, 0.02)[:100]
print(np.shape(d_test))
max_seq_length = t_test.shape[1]
# d_test, t_test = d_test[:100], t_test[:100]

new_data = torch.tensor(d_test, dtype=torch.float32).to(device)

# Load the saved model
model = CNNLSTMModel(max_seq_length).to(device)
model.load_state_dict(torch.load("torch_model.pth"))
model.eval()

# Run the model on the new dataset
with torch.no_grad():
    outputs = model(new_data)

# Convert the outputs to a NumPy array
outputs = outputs.cpu().numpy()

# Print the first few predictions
print("Model predictions on new dataset:")
print(outputs)

/home/thomas/Documents/ActiveAlarm_Processing/importData.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trimmed_raw_df["time"] = trimmed_raw_df["time"].apply(lambda x: x - timestamps_df["startTime"][0])
/home/thomas/Documents/ActiveAlarm_Processing/importData.py:161: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_time_dataset["time"] = start_time_dataset["time"] - start_time


[0.0000e+00 2.0000e-02 4.0000e-02 ... 1.3294e+02 1.3296e+02 1.3298e+02]
(6395, 10, 1000, 1)


/tmp/ipykernel_7861/2575147498.py:51: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("torch_model.pth"))


OutOfMemoryError: CUDA out of memory. Tried to allocate 3.04 GiB. GPU 0 has a total capacity of 7.90 GiB of which 1.33 GiB is free. Process 7711 has 2.93 GiB memory in use. Including non-PyTorch memory, this process has 3.38 GiB memory in use. Of the allocated memory 551.19 MiB is allocated by PyTorch, and 2.75 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [10]:
import torch
import gc

gc.collect()

torch.cuda.empty_cache()
print(torch.cuda.memory_summary(device=None, abbreviated=False))

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 5            |        cudaMalloc retries: 5         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 282210 KiB |   3392 MiB |   4655 MiB |   4380 MiB |
|       from large pool | 281741 KiB |   3392 MiB |   4650 MiB |   4375 MiB |
|       from small pool |    469 KiB |      1 MiB |      5 MiB |      5 MiB |
|---------------------------------------------------------------------------|
| Active memory         | 282210 KiB |   3392 MiB |   4655 MiB |   4380 MiB |
|       from large pool | 281741 KiB |   3392 MiB |   4650 MiB |

In [7]:
print(torch.cuda.is_available())
print(torch.__version__)

True
2.4.1+cu124
